In [75]:
import torch
from stable_baselines3 import PPO
import gymnasium as gym
import numpy as np
import imageio

In [76]:
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        # Access relevant state and action variables
        
        #state
        z = obs[0]                  # Height of the robot
        a = obs[1]                  # Angle of the robot
        a_hip = obs[2]              # Angle of the hip
        a_knee = obs[3]             # Angle of the knee
        a_ankle = obs[4]            # Angle of the ankle
        v_x = obs[5]                # Velocity in x direction
        v_z = obs[6]                # Velocity in z direction
        a_d = obs[7]                # Angular velocity 
        a_hip_d = obs[8]            # Angular velocity of the hip
        a_knee_d = obs[9]           # Angular velocity of the knee
        a_ankle_d = obs[10]         # Angular velocity of the ankle
        
        #action
        torque_hip = action[0]      # Torque applied to the hip
        torque_knee = action[1]     # Torque applied to the knee
        torque_ankle = action[2]    # Torque applied to the ankle

        #different criteria for reward
        energy_used = np.sum(np.square(action))  # Simplistic energy calculation

        # Custom reward logic
        custom_reward = z * 1.0                    # Reward for height     
        custom_reward += a_d * 0.5                 # Reward for angular spinning
        custom_reward -= energy_used * 0.1         # Penalize energy consumption

        # used by openai
        # backroll = -obs[7]
        # height = obs[0]
        # vel_act = action[0] * obs[8] + action[1] * obs[9] + a[2] * obs[10]
        # backslide = -obs[5]
        # reward = backroll * (1.0 + .3 * height + .1 * vel_act + .05 * backslide)

        if done:
            custom_reward -= 10.0  # Heavy penalty for falling

        return obs, custom_reward, done, truncated, info

In [77]:
# Use the wrapper with the Hopper environment
env = gym.make('Hopper-v4', render_mode='rgb_array')
env = CustomRewardWrapper(env)

model = PPO("MlpPolicy", env, verbose=1)

# Train the model
n_learning_steps = 1000000
model.learn(total_timesteps=n_learning_steps)

# Save the model
model.save("hopper")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.6     |
|    ep_rew_mean     | -1.87    |
| time/              |          |
|    fps             | 349      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25          |
|    ep_rew_mean          | 2.91        |
| time/                   |             |
|    fps                  | 245         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017887086 |
|    clip_fraction        | 0.239       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [81]:
vec_env = model.get_env()
obs = vec_env.reset()

writer = imageio.get_writer('hopper-flip.mp4', fps=100)


N_step = 100000
for i in range(N_step):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    writer.append_data(vec_env.render("rgb_array"))
    #VecEnv resets automatically
    if done:
      print("Episode finished after {} timesteps".format(i+1))
      break
      obs = vec_env.reset()

writer.close()

Episode finished after 1000 timesteps
